# pydantic-yaml-parser

> Generate human-readable error messages when loading yaml files into pydantic

[![](https://github.com/hamelsmu/pydantic-yaml-parser/actions/workflows/test.yaml/badge.svg)](https://github.com/hamelsmu/pydantic-yaml-parser/actions/workflows/test.yaml)

## Install

```sh
pip install pydantic_yaml_parser
```

## Background & Usage

### 1. Loading YAML into Pydantic models

To load YAML into a Pydantic model, you have to first convert the YAML string to a dict.  For example, given the below yaml file:

In [ ]:
yaml_string="""
setting_1: Hello
setting_2:
    - kind: name
      sublist:
          - first
          - second
"""

You might define a Pydantic model to represent it like this:

In [ ]:
from pydantic import BaseModel
from typing import List

class Setting2(BaseModel):
    kind: str
    sublist: List

class Test(BaseModel):
    setting_1: str
    setting_2: List[Setting2]

You can load the yaml file into a dict, and into the Pydantic model like so using [pyyaml](https://pyyaml.org/wiki/PyYAMLDocumentation):

In [ ]:
import yaml
yml_dict = yaml.safe_load(yaml_string)

# use `parse_obj` to load a dict
Test.parse_obj(yml_dict)

Test(setting_1='Hello', setting_2=[Setting2(kind='name', sublist=['first', 'second'])])

### 2. The Pydantic validation error message

However, let's say there is an error in your yaml file such that you accidentally set `sublist` to `false`, instead of setting `sublist` to `list` type, you will get an error message that looks like this:

In [ ]:
yaml_string="""
setting_1: ok
setting_2:
    - kind: name
      sublist: false
"""
yaml_dict_error = yaml.safe_load(yaml_string)

In [ ]:
#|eval: false
Test.parse_obj(yaml_dict_error)

ValidationError: 1 validation error for Test
setting_2 -> 0 -> sublist
  value is not a valid list (type=type_error.list)

This error message is a bit confusing, especially for those with no prior experience with pydantic! 

### 3. Human readable error messages with `pydantic_yaml_parser`

When you use `pydantic_yaml_parser` you get an error message that is much clearer:

In [ ]:
from pydantic_yaml_parser.yaml import YamlModel

class Test(YamlModel):
    setting_1: str
    setting_2: List[Setting2]

In [ ]:
#|eval: false
Test.from_dict(yaml_dict_error)

ValueError: Configuration error(s) for YAML:
 - value is not a valid list: `sublist` for element 0 in the list for `setting_2`

## Further Reading

For more examples of error validation see [these docs](00_yaml.ipynb#error-validation).